In [3]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, MaxPooling2D,
    Dropout, GlobalAveragePooling2D, Dense
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
)
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load and preprocess data
(x_train_full, y_train_full), (x_test, y_test) = mnist.load_data()

# Reshape and normalize
x_train_full = np.expand_dims(x_train_full, -1).astype('float32') / 255
x_test = np.expand_dims(x_test, -1).astype('float32') / 255

# Convert labels to one-hot encoding
y_train_full = to_categorical(y_train_full, 10)
y_test = to_categorical(y_test, 10)

# Train-validation split
x_train, x_val, y_train, y_val = train_test_split(
    x_train_full, y_train_full, test_size=0.2, random_state=42
)

# Data augmentation generator (reduced augmentation for speed)
datagen = ImageDataGenerator(
    rotation_range=8,  # Reduced from 10
    width_shift_range=0.08,  # Reduced from 0.1
    height_shift_range=0.08,  # Reduced from 0.1
    zoom_range=0.08  # Reduced from 0.1
)
datagen.fit(x_train)

# Build optimized CNN model with reduced complexity
model = Sequential([
    # First block - keep initial filters for good feature extraction
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(28,28,1)),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.25),
    
    # Second block - reduced filters from 64 to 48
    Conv2D(48, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.25),
    
    # Third block - reduced filters from 128 to 64, removed one Conv2D layer
    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    GlobalAveragePooling2D(),
    Dropout(0.5),
    
    Dense(10, activation='softmax')
])

# Compile with slightly higher learning rate for faster convergence
model.compile(
    optimizer=Adam(learning_rate=0.0003),  # Increased from 0.0001
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks with adjusted parameters for faster training
callbacks = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),  # Reduced from 10
    ModelCheckpoint('best_model.h5', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, min_lr=1e-6)  # Increased factor from 0.2
]

# Train model with larger batch size
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=128),  # Increased from 64
    steps_per_epoch=len(x_train) // 128,
    epochs=50,  # Reduced from 100
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=1
)

# Save final model
model.save('mnist_cnn_final.h5')

/home/maruf/.cache/pypoetry/virtualenvs/pattern-recognition-neural-network-coursew-8OHAYx0u-py3.10/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - accuracy: 0.4379 - loss: 1.6909

375/375 ━━━━━━━━━━━━━━━━━━━━ 81s 202ms/step - accuracy: 0.4383 - loss: 1.6900 - val_accuracy: 0.2264 - val_loss: 4.5073 - learning_rate: 3.0000e-04
Epoch 2/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 0.8434 - loss: 0.6968

375/375 ━━━━━━━━━━━━━━━━━━━━ 73s 193ms/step - accuracy: 0.8434 - loss: 0.6966 - val_accuracy: 0.6115 - val_loss: 1.0390 - learning_rate: 3.0000e-04
Epoch 3/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.9062 - loss: 0.4154

375/375 ━━━━━━━━━━━━━━━━━━━━ 70s 186ms/step - accuracy: 0.9063 - loss: 0.4153 - val_accuracy: 0.9413 - val_loss: 0.2236 - learning_rate: 3.0000e-04
Epoch 4/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step - accuracy: 0.9272 - loss: 0.3039

...

375/375 ━━━━━━━━━━━━━━━━━━━━ 67s 180ms/step - accuracy: 0.9783 - loss: 0.0754 - val_accuracy: 0.9915 - val_loss: 0.0295 - learning_rate: 9.0000e-05
Epoch 26/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 81s 177ms/step - accuracy: 0.9799 - loss: 0.0700 - val_accuracy: 0.9916 - val_loss: 0.0302 - learning_rate: 9.0000e-05
Epoch 27/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - accuracy: 0.9787 - loss: 0.0720

375/375 ━━━━━━━━━━━━━━━━━━━━ 67s 178ms/step - accuracy: 0.9787 - loss: 0.0720 - val_accuracy: 0.9918 - val_loss: 0.0281 - learning_rate: 9.0000e-05
Epoch 28/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 67s 177ms/step - accuracy: 0.9779 - loss: 0.0741 - val_accuracy: 0.9909 - val_loss: 0.0303 - learning_rate: 9.0000e-05
Epoch 29/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 68s 182ms/step - accuracy: 0.9789 - loss: 0.0719 - val_accuracy: 0.9912 - val_loss: 0.0286 - learning_rate: 9.0000e-05
Epoch 30/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 70s 187ms/step - accuracy: 0.9805 - loss: 0.0666 - val_accuracy: 0.9920 - val_loss: 0.0285 - learning_rate: 9.0000e-05
Epoch 31/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 70s 187ms/step - accuracy: 0.9811 - loss: 0.0683 - val_accuracy: 0.9920 - val_loss: 0.0292 - learning_rate: 2.7000e-05
Epoch 32/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 70s 187ms/step - accuracy: 0.9797 - loss: 0.0687 - val_accuracy: 0.9920 - val_loss: 0.0285 - learning_rate: 2.7000e-05
Epoch 33/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/st

375/375 ━━━━━━━━━━━━━━━━━━━━ 71s 188ms/step - accuracy: 0.9810 - loss: 0.0659 - val_accuracy: 0.9919 - val_loss: 0.0279 - learning_rate: 2.7000e-05
Epoch 34/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 71s 188ms/step - accuracy: 0.9801 - loss: 0.0654 - val_accuracy: 0.9921 - val_loss: 0.0282 - learning_rate: 2.7000e-05
Epoch 35/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.9808 - loss: 0.0677

375/375 ━━━━━━━━━━━━━━━━━━━━ 71s 189ms/step - accuracy: 0.9808 - loss: 0.0677 - val_accuracy: 0.9919 - val_loss: 0.0274 - learning_rate: 2.7000e-05
Epoch 36/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 71s 189ms/step - accuracy: 0.9811 - loss: 0.0653 - val_accuracy: 0.9922 - val_loss: 0.0279 - learning_rate: 2.7000e-05
Epoch 37/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 71s 188ms/step - accuracy: 0.9807 - loss: 0.0669 - val_accuracy: 0.9925 - val_loss: 0.0275 - learning_rate: 2.7000e-05
Epoch 38/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - accuracy: 0.9814 - loss: 0.0667

375/375 ━━━━━━━━━━━━━━━━━━━━ 70s 185ms/step - accuracy: 0.9814 - loss: 0.0667 - val_accuracy: 0.9923 - val_loss: 0.0272 - learning_rate: 2.7000e-05
Epoch 39/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 71s 189ms/step - accuracy: 0.9779 - loss: 0.0717 - val_accuracy: 0.9923 - val_loss: 0.0274 - learning_rate: 2.7000e-05
Epoch 40/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 71s 188ms/step - accuracy: 0.9798 - loss: 0.0680 - val_accuracy: 0.9919 - val_loss: 0.0273 - learning_rate: 2.7000e-05
Epoch 41/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 69s 183ms/step - accuracy: 0.9793 - loss: 0.0688 - val_accuracy: 0.9924 - val_loss: 0.0273 - learning_rate: 2.7000e-05
Epoch 42/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 69s 183ms/step - accuracy: 0.9801 - loss: 0.0686 - val_accuracy: 0.9923 - val_loss: 0.0273 - learning_rate: 8.1000e-06
Epoch 43/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 69s 184ms/step - accuracy: 0.9803 - loss: 0.0666 - val_accuracy: 0.9925 - val_loss: 0.0273 - learning_rate: 8.1000e-06
Epoch 44/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 74s 198ms/s